In [1]:
#https://shotanuki.com/python%E3%81%A7pdf%E3%82%92%E7%94%BB%E5%83%8F%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B/
#PythonでPDFを画像に変換する
from pdf2image import convert_from_path
import datetime
import os
now = datetime.date.today()

# カレントディレクトリの全画像をループする
for pdfname in os.listdir('./mergedTRF/'):# ❶
    ext = os.path.splitext(pdfname)
    #if ext == '.pdf':
    images = convert_from_path('./mergedTRF/'+pdfname)
    os.makedirs('./trf/'+str(now), exist_ok=True)
    i = 0
    for image in images:
        image.save('./trf/'+str(now) +'/trf_' + str(pdfname)+'_'+str(i)+'.png', 'png')
        i += 1
       

In [2]:

eng = [
    (620,700,525,1030), #Pt Last
    (685,760,525,1030), #Pt First
    (495,560,1095,1600), #Dr Last
    (550,620,1095,1600), #Dr First
    ]

%matplotlib inline
from keras.models import load_model
import numpy as np
import cv2
import csv
import matplotlib.pyplot as plt
from PIL import Image
import sys,os
import datetime
now = datetime.date.today()
now_time = datetime.datetime.today()
now_time = str(now_time.hour)+'-'+str(now_time.minute)+'-'+str(now_time.second)

os.makedirs('./trf/'+str(now), exist_ok=True)

os.makedirs('./english/'+ str(now), exist_ok = True)
os.makedirs('./english/' + str(now) + '/A', exist_ok = True)
os.makedirs('./english/' + str(now) + '/B', exist_ok = True)
os.makedirs('./english/' + str(now) + '/C', exist_ok = True)
os.makedirs('./english/' + str(now) + '/D', exist_ok = True)
os.makedirs('./english/' + str(now) + '/E', exist_ok = True)
os.makedirs('./english/' + str(now) + '/F', exist_ok = True)
os.makedirs('./english/' + str(now) + '/G', exist_ok = True)
os.makedirs('./english/' + str(now) + '/H', exist_ok = True)
os.makedirs('./english/' + str(now) + '/I', exist_ok = True)
os.makedirs('./english/' + str(now) + '/J', exist_ok = True)
os.makedirs('./english/' + str(now) + '/K', exist_ok = True)
os.makedirs('./english/' + str(now) + '/L', exist_ok = True)
os.makedirs('./english/' + str(now) + '/M', exist_ok = True)
os.makedirs('./english/' + str(now) + '/N', exist_ok = True)
os.makedirs('./english/' + str(now) + '/O', exist_ok = True)
os.makedirs('./english/' + str(now) + '/P', exist_ok = True)
os.makedirs('./english/' + str(now) + '/Q', exist_ok = True)
os.makedirs('./english/' + str(now) + '/R', exist_ok = True)
os.makedirs('./english/' + str(now) + '/S', exist_ok = True)
os.makedirs('./english/' + str(now) + '/T', exist_ok = True)
os.makedirs('./english/' + str(now) + '/U', exist_ok = True)
os.makedirs('./english/' + str(now) + '/V', exist_ok = True)
os.makedirs('./english/' + str(now) + '/W', exist_ok = True)
os.makedirs('./english/' + str(now) + '/X', exist_ok = True)
os.makedirs('./english/' + str(now) + '/Y', exist_ok = True)
os.makedirs('./english/' + str(now) + '/Z', exist_ok = True)
os.makedirs('./english/' + str(now) + '/Error', exist_ok = True)

os.makedirs('./results/english_results/' + str(now), exist_ok = True)
os.makedirs('./csv/'+ str(now), exist_ok = True)
path_w = './csv/'+ str(now)+'/english_'+str(now)+'_'+str(now_time)+'.csv'

labels = ["A ", "B ", "C ", "D ", "E ", "F ", "G ", "H ", \
          "I ", "J ", "K ", "L ", "M ", "N ", "O ", "P ", \
          "Q ", "R ", "S ", "T ", "U ", "V ", "W ", "X ", "Y ", "Z ","Error"]

model = load_model('english_CNN_model.h5')
model.load_weights('english_CNN_weight.h5')


Using TensorFlow backend.


In [3]:
# 依頼書画像から番号領域を抽出する関数
def detect_PtLastName(fname):
    # 画像を読み込む
    nums = [
    (620,700,525,1030), #Pt Last
    (685,760,525,1030), #Pt First
    (495,560,1095,1600), #Dr Last
    (550,620,1095,1600), #Dr First   
    ]
    for j, pt in enumerate(eng):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
            

In [4]:
# 依頼書画像から番号領域を抽出する関数
def detect_PtFirstName(fname):
    # 画像を読み込む
    nums = [
    #(620,700,525,1030), #Pt Last
    (685,760,525,1030), #Pt First
    (495,560,1095,1600), #Dr Last
    (550,620,1095,1600), #Dr First     
    ]
    for j, pt in enumerate(eng):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
            

In [5]:
# 依頼書画像から番号領域を抽出する関数
def detect_DrLastName(fname):
    # 画像を読み込む
    nums = [
    #(620,700,525,1030), #Pt Last
    #(685,760,525,1030), #Pt First
    (495,560,1095,1600), #Dr Last
    (550,620,1095,1600), #Dr First     
    ]
    for j, pt in enumerate(eng):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
     

In [6]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DrFirstName(fname):
    # 画像を読み込む
    nums = [
    #(620,700,525,1030), #Pt Last
    #(685,760,525,1030), #Pt First
    #(495,560,1095,1600), #Dr Last
    (550,620,1095,1600), #Dr First   
    ]
    for j, pt in enumerate(eng):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
     

In [7]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_PtLastName('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'PtLastName'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(25,25))

                img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
                img2 = img2.reshape(25 * 25 *1),astype('float32') /255
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./english/'+str(now)+'/'+str(labels[idx])+'/PtLastName_'+ str(j)+'_'+str(labels[idx])+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/english_results/'+str(now)+'PtLastName_'+ str(j)+'_'+ filename, img)
                if idx == 27: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

NameError: name 'astype' is not defined

In [ ]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_PtFirstName('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                #f.write(str(filename)+'\t'+'PtFirstName'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.imread(img2)
                img2 = cv2.resize(img2,(25,25))
                img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./english/'+str(now)+'/'+str(labels[idx])+'/PtFirstName_'+ str(j)+'_'+str(labels[idx])+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/english_results/'+str(now)+'/PtFirstName_'+ str(j)+'_'+ filename, img)
                if idx == 27: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [ ]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DrLastName('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DrLastName'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.imread(img2)
                img2 = cv2.resize(img2,(25,25))
                img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./english/'+str(now)+'/'+str(labels[idx])+'/DrLastName_'+ str(j)+'_'+str(labels[idx])+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/english_results/'+str(now)+'/DrLastName_'+ str(j)+'_'+ filename, img)
                if idx == 27: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')
                    

In [ ]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DrFirstName('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DrFirstName'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = cv2.imread(img2)
                img2 = cv2.resize(img2,(25,25))
                img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./english/'+str(now)+'/'+str(labels[idx])+'/DrFirstName_'+ str(j)+'_'+str(labels[idx])+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/english_results/'+str(now)+'/DrFirstName_'+ str(j)+'_'+ filename, img)
                if idx == 27: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')
                    

In [ ]:
import shutil
shutil.move('./english/' + str(now)+'/','./english/'+str(now)+'_'+ str(now_time)+'/')
shutil.move('./results/english_results/' + str(now) + '/','./results/english_results/' + str(now) + '_'+str(now_time)+'/')
shutil.move('./trf/'+str(now)+'/','./finished_trf/english/'+str(now)+'_'+str(now_time)+'/')